In [11]:
import os

dataDir = '../../example_dataset/raw/'

listFiles = os.listdir(dataDir)
listFiles

['park_ss_ff_0MPa_000315.edf.ge5',
 'dark_after_000313.edf.ge5',
 'dark_before_000320.edf.ge5']

In [12]:
darkFilesList = []
darkFilesDict = {}

for fileString in listFiles:
    if fileString.startswith('dark_before'):
        # find the index of each dark before
        idx = int(fileString[12:18])
        #print(int(fileString[12:18]))
        darkFilesDict[idx] = fileString

darkFilesDict

{320: 'dark_before_000320.edf.ge5'}

In [13]:
darkFilesDict

{320: 'dark_before_000320.edf.ge5'}

In [14]:
darkFilesKeys = sorted(darkFilesDict)
darkFilesKeys

[320]

In [15]:
parkFilesList = []
parkFilesDict4Idx = {}

# park_ss_ff

for fileString in listFiles:
    if fileString.startswith('park_ss_ff'):
        # find the index of each file
        x = fileString.split("_")
        idx = int(x[-1].split(".")[0])
        parkFilesDict4Idx[fileString] = idx
#         idx = int(fileString[12:18])
#         #print(int(fileString[12:18]))
#         darkFilesDict[idx] = fileString
#         darkFiles.append(fileString)

# 
parkFilesDict4Idx 

{'park_ss_ff_0MPa_000315.edf.ge5': 315}

In [16]:
# next is to find the dark files for each file
darkFilesDict4Idx = {}

# park_ss_ff

for fileString in listFiles:
    if fileString.startswith('park_ss_ff'):
        # find the index of each file
        x = fileString.split("_")
        idx = int(x[-1].split(".")[0])
        # next is to find the corresponding dark file
        for darkKey in darkFilesKeys:
            if darkKey >= idx:
                print(str(idx) + "-" + str(darkKey))
                darkFilesDict4Idx[fileString] = darkFilesDict[darkKey]
                break

darkFilesDict4Idx

315-320


{'park_ss_ff_0MPa_000315.edf.ge5': 'dark_before_000320.edf.ge5'}

In [17]:
# now start to convert ge5 files to patches
from src.datasets.ge2patch import *

In [19]:
outDir = '../../example_dataset/test_patch_folder/'

i = 1

for key in darkFilesDict4Idx:
    print(f"Reading dark file from {dataDir+darkFilesDict4Idx[key]} ... ")
    dark = ge_raw2array(dataDir+darkFilesDict4Idx[key], skip_frm=0).mean(axis=0).astype(np.float32)
    print(f"Done with reading dark file from {dataDir+darkFilesDict4Idx[key]} with {dark}")
    print(f"The input file is {key}")
    outFile = outDir + key[:-3] + "h5"
    print(f"The output file is {outFile}")
    print(f"Start to generate patches for file {key} ({i}/{len(darkFilesDict4Idx)})...")
    ge_raw2patch(gefname=dataDir+key, ofn=outFile, dark=dark, 
                 bkgd=100, psz=15, skip_frm=0, min_intensity=0, max_r=None)
    print(f"Done generating patches for file {key} ({i}/{len(darkFilesDict4Idx)})...")
    i += 1

Reading dark file from ../../example_dataset/raw/dark_before_000320.edf.ge5 ... 
Done with reading dark file from ../../example_dataset/raw/dark_before_000320.edf.ge5 with [[1650.  1658.3 1656.7 ... 1834.6 1833.4 1829.5]
 [1669.9 1674.9 1672.5 ... 1825.7 1823.8 1823.2]
 [1678.2 1682.4 1679.8 ... 1834.1 1833.8 1830.6]
 ...
 [1678.5 1679.1 1681.8 ... 1736.2 1736.3 1734.8]
 [1672.6 1670.8 1673.  ... 1730.7 1730.  1726.8]
 [1635.5 1635.6 1633.9 ... 1701.5 1700.3 1699.2]]
The input file is park_ss_ff_0MPa_000315.edf.ge5
The output file is ../../example_dataset/test_patch_folder/park_ss_ff_0MPa_000315.edf.h5
Start to generate patches for file park_ss_ff_0MPa_000315.edf.ge5 (1/1)...
47697 patches of size 15 cropped from ../../example_dataset/raw/park_ss_ff_0MPa_000315.edf.ge5, 0 are too big.
Done generating patches for file park_ss_ff_0MPa_000315.edf.ge5 (1/1)...
